In [158]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


/Users/uditagupta93/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text', 'sample']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [159]:
test_df = pd.read_csv('test.csv')
test_df.head(5)

,ID,WRONG
0,0,contenpted
1,1,begining
2,2,problam
3,3,dirven
4,4,exstacy


In [160]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488665

In [161]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [162]:
WORDS = tokens(TEXT)

In [163]:
def sample(bag, n=10):
    return ' '.join(random.choice(bag) for _ in range(n))

In [164]:
sample(WORDS)

'shook of civil many at his permanently became arizona life'

In [165]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [166]:
def load_counts(filename, sep='\t'):
    """Return a Counter initialized from key-value pairs, 
    one on each line of filename."""
    C = Counter()
    for line in open(filename):
        key, count = line.split(sep)
        C[key] = int(count)
    return C


COUNTS1 = load_counts('count_1w.txt')

In [167]:
COUNTS_new= COUNTS+COUNTS1
print(COUNTS_new.most_common(10))

[('the', 23135931192), ('of', 13151982801), ('and', 12997676279), ('to', 12137009624), ('a', 9081195853), ('in', 8469427021), ('for', 5933328650), ('is', 4705753590), ('on', 3750429842), ('that', 3400043615)]


In [168]:
COUNTS_new_dict={}
COUNTS_new_dict = dict(COUNTS_new)

In [169]:
error_counter =0
error_clash =[]
for key in test_df['WRONG']:
    if key in COUNTS_new_dict:
        error_counter += 1
        error_clash.append(key)
        COUNTS_new_dict.pop(key, None)
        #del COUNTS_new[key]
        
#print(error_counter)

COUNTS_new1 = Counter(dict(COUNTS_new_dict))  # convert from a list of (elem, cnt) pairs
#COUNTS_new_dict['begining']
#del COUNTS_new['speling']
COUNTS_new1['begining']

0

# Spell Checker

In [170]:
def correct(word):
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    #print(candidates)
    #print(COUNTS.get)
    #print(max(candidates, key=COUNTS.get))
    #return(candidates)
    return max(candidates, key=COUNTS_new1.get)

In [171]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS_new1}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [172]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [173]:
list(map(correct, tokens('begining')))

['beginning']

In [174]:
test_df.drop('ID', axis=1)
test_list = test_df['WRONG'].tolist()
#print(test_list)

In [175]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [176]:
correct_list =[]
for text in test_list:
    correct_list.append(correct_text(text))
    
#print(correct_list)

In [177]:
import csv

csvfile = "sample_test_submit.csv"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in correct_list:
        writer.writerow([val])  